<a href="https://colab.research.google.com/github/raguram/eva/blob/master/S4/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torchsummary

In [0]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm_notebook as tqdm
import torch.optim as optim
from sklearn.metrics import confusion_matrix

In [7]:
cpu = torch.device("cpu")
torch.manual_seed(1)

def isCuda():
  return torch.cuda.is_available()

def getDevice(): 
  return torch.device("cuda" if isCuda() else "cpu")

def transformations(): 
  # 1. Create a tensor from the input
  return transforms.Compose([transforms.ToTensor()])

def MNIST(isTrain=True, transforms=transformations()): 
  return datasets.MNIST("../data", isTrain, transform=transforms, download=True)

def dataLoader(dataset): 
  args = {'num_workers': 1, 'pin_memory': True} if isCuda() else {}
  return torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, **args)

train_data = MNIST()
train_loader = dataLoader(train_data)

test_data = MNIST(isTrain=False)
test_loader = dataLoader(test_data)

print(f'\nSize of train data:{train_data.data.shape}')
print(f'\nSize of test data:{test_data.data.shape}')


Size of train data:torch.Size([60000, 28, 28])

Size of test data:torch.Size([10000, 28, 28])


In [8]:
class Net(nn.Module): 

  def __init__(self): 
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    self.pool1 = nn.MaxPool2d(2, 2)

    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
    self.pool2 = nn.MaxPool2d(2, 2)

    self.conv5 = nn.Conv2d(256, 512, 3)
    self.conv6 = nn.Conv2d(512, 1024, 3)
    self.conv7 = nn.Conv2d(1024, 10, 3)

  def forward(self, x): 

    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    x = x.view(-1, 10)
    return F.log_softmax(x, dim=1)

  def summarize(self, input): 
    summary(self, input_size=input)

Net().to(getDevice()).summarize((1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

In [0]:
def compute_accuracy(prediction, target): 
  return 100 * prediction.eq(target.view_as(prediction)).sum().item() / len(prediction) 

def compute_metrics(data, prediction, target): 
  corrects = prediction.eq(target.view_as(prediction))
  accuracy = corrects.sum().item() / len(prediction)
  miss_indices = ~corrects
  miss_data = data[miss_indices],  
  miss_predictions = prediction[miss_indices]
  miss_targets = target[miss_indices]
  print(confusion_matrix(target.to(cpu), prediction.to(cpu)))
  return accuracy * 100

In [0]:
def train_one_batch(model, data, target, optimizer): 
  optimizer.zero_grad()
  output = model(data)
  loss = F.nll_loss(output, target)
  loss.backward()
  optimizer.step()
  return (loss, output.argmax(dim=1))

def train_one_epoch(model, train_loader, optimizer, device=getDevice()): 
  model.train()
  pbar = tqdm(train_loader, ncols=1000) 
  wholePred = []
  wholeData = []
  wholeTarget = []
  for idx, (data, target) in enumerate(pbar): 
    data, target = data.to(device), target.to(device)
    (loss, prediction) = train_one_batch(model, data, target, optimizer)
    accuracy = compute_accuracy(prediction, target)
    pbar.set_description(desc= f'Loss={loss.item()}\t id={idx}\t accuracy={accuracy}')
    wholePred.append(prediction)
    wholeData.append(data)
    wholeTarget.append(target)
  return (torch.cat(wholeData), torch.cat(wholeTarget), torch.cat(wholePred))

def train(model, train_loader, optimizer, epoch): 
  for e in range(0, epoch): 
    (train_data, train_target, train_prediction) = train_one_epoch(model, train_loader, optimizer)
    print(f'\nTrain Accuracy for epoch:{e} is {compute_accuracy(train_prediction, train_target)}%')
    

In [0]:
def test_one_batch(model, data, target): 
  output = model(data)
  loss = F.nll_loss(output, target)  
  return (loss, output.argmax(dim=1))
  
def test(model, test_loader, device=getDevice()): 
  model.eval()
  pbar = tqdm(test_loader, ncols=1000)
  wholePred = []
  wholeData = []
  wholeTarget = []
  with torch.no_grad(): 
    for idx, (data, target) in enumerate(pbar): 
      data, target = data.to(device), target.to(device)
      (loss, prediction) = test_one_batch(model, data, target)
      accuracy = compute_accuracy(prediction, target)
      pbar.set_description(desc= f'Loss={loss.item()}\t id={idx}\t accuracy={accuracy}%')
      wholePred.append(prediction)
      wholeData.append(data)
      wholeTarget.append(target)
  return (torch.cat(wholeData), torch.cat(wholeTarget), torch.cat(wholePred))

In [12]:
net = Net().to(getDevice())
optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)
train(net, train_loader, optimizer, 20)



Train Accuracy for epoch:0 is 80.52166666666666%




Train Accuracy for epoch:1 is 97.985%




Train Accuracy for epoch:2 is 98.71666666666667%




Train Accuracy for epoch:3 is 99.075%




Train Accuracy for epoch:4 is 99.21666666666667%


...



Train Accuracy for epoch:15 is 99.92833333333333%




Train Accuracy for epoch:16 is 99.93166666666667%




Train Accuracy for epoch:17 is 99.97833333333334%




Train Accuracy for epoch:18 is 99.97%




Train Accuracy for epoch:19 is 99.99%


In [0]:
(test_data, test_target, test_prediction) = test(net, test_loader)
print(f'Test Accuracy = {compute_metrics(test_data, test_prediction, test_target)}%')

[[ 978    0    0    0    0    0    0    1    1    0]
 [   0 1133    1    0    0    0    1    0    0    0]
 [   0    0 1027    0    0    0    1    3    1    0]
 [   0    0    1 1003    0    3    0    0    3    0]
 [   0    0    0    0  974    0    3    0    0    5]
 [   1    0    0    8    0  880    1    0    1    1]
 [   2    2    0    0    1    2  949    0    1    1]
 [   0    3    4    0    0    0    0 1018    2    1]
 [   3    0    0    1    0    2    0    0  966    2]
 [   0    0    0    0    7    2    0    4    1  995]]
Test Accuracy = 99.22999999999999%
